##### JOSE MARIA GOMEZ PULIDO    
##### EVA SANCHEZ MUÑOZ
##### ALVARO VELASCO DIAZ

In [10]:
%%writefile waveformOsc.py
#Ejercicio 1

from consts import *
import numpy as np         
import sounddevice as sd       
import matplotlib.pyplot as plt
import scipy.signal as signal
from osc import *
from oscFM import *

class WaveformOsc(Osc):
    def __init__(self,freq=440.0,amp=1.0,phase=0.0, waveform="Sine"):
        Osc(freq,amp,phase)
        self.waveform = waveform

    def next(self):    
        arange = 2*np.pi*np.arange(self.frame,self.frame+CHUNK)*self.freq/SRATE
        out = None
        if self.waveform == "Sine":
            out = self.amp*np.sin(arange)
        elif self.waveform == "Square":
            out = self.amp * signal.square(arange)
        elif self.waveform == "Triangle":
            out = self.amp * signal.sawtooth(arange,0.5)
        elif self.waveform == "Sawtooth":
            out = self.amp * signal.sawtooth(arange)
        self.frame += CHUNK
        return out

    def setWaveForm(self, form):
        self.waveform = form

Overwriting waveformOsc.py


In [11]:
%%writefile waveformOscFM.py
from consts import *
import numpy as np         
import sounddevice as sd       
import matplotlib.pyplot as plt
import scipy.signal as signal
from waveformOsc import *
from oscFM import *

class WaveformOscFM(OscFM):
    def __init__(self,fc=110.0,amp=1.0,fm=6.0, beta=1.0, carrier="Sine", mod="Sine"):
        OscFM(fc,amp,fm,beta)
        # moduladora = βsin(2πfm)
        self.carrier = carrier
        self.mod = WaveformOsc(freq=fm,amp=beta,waveform=mod)
        
    def next(self):  
        # sin(2πfc+mod)  
        # sacamos el siguiente chunk de la moduladora
        mod = self.mod.next()

        # soporte para el chunk de salida
        sample = np.arange(self.frame,self.frame+CHUNK)  
        sample = 2*np.pi*self.fc*sample/SRATE +mod     
        # aplicamos formula
        if self.carrier == "Sine":
            out =  self.amp*np.sin(sample)
        elif self.carrier == "Square":
            out = self.amp*signal.square(sample)
        elif self.carrier == "Triangle":
            out = self.amp * signal.sawtooth(sample,0.5)
        elif self.carrier == "Sawtooth":
            out = self.amp * signal.sawtooth(sample)
        self.frame += CHUNK
        return out 

    def setWaveforms(self, carrier, mod):
        self.carrier = carrier
        self.mod.setWaveForm(mod)

Writing waveformOscFM.py


In [13]:
%%writefile waveformSynthFM.py
import numpy as np   
import matplotlib.pyplot as plt
from consts import *
from tkinter import *
from adsr import *
from waveformOscFM import *
from synthFM import *
class WaveformSynthFM(SynthFM):
    def __init__(self,
                fc=110,amp=1.0,ratio=0.5, beta=5.0,   # parámetros del generador FM
                attack=0.01,decay=0.02, sustain=0.3,release=1.0, carrier="Sine", mod = "Sine"): # parámetros del ADSR   
        SynthFM(fc,amp,ratio,beta,attack,decay,sustain,release)     
        self.signal = WaveformOscFM(self.fc,self.amp,self.fm,self.beta,carrier,mod) # generador
        

    def start(self):
        self.adsr.start()

    # siguiente chunk del generador
    def next(self): 
        out = self.signal.next()*self.adsr.next()
        if self.adsr.state=='off': # cuando acaba el adsr por completo (incluido el release)
            self.state = 'off'     # el sinte tb acaba de producir señal
        return out     
    
    # el noteOff del sinte activa el release del ADSR
    def noteOff(self):
        self.adsr.release()

    def setAmp(self,val): 
        self.amp = val 

    def setFm(self,val): 
        self.fm = val  

    def setBeta(self,val): 
        self.beta = val

    def setFMWaveform(self,carrier,mod):
        self.signal.setWaveforms(carrier,mod)

Overwriting waveformSynthFM.py


In [14]:
%%writefile waveformInstrument.py
from instrument import *
from waveformSynthFM import *
from tkinter import * 

class WaveformInstrument(Instrument):
    def __init__(self,tk,name="FM synthetizer",amp=0.2,ratio=3,beta=0.6): 
        Instrument(tk,name,amp,ratio,beta)
    
    def noteOn(self,midiNote):
        # si está el dict de canales reactivamos synt -> reiniciamos adsr del synt
        if midiNote in self.channels:       
            self.channels[midiNote].start() 

        # si no está, miramos frecuencia en la tabla de frecuencias
        # y generamos un nuevo synth en un canal indexado con notaMidi
        # con los parámetros actuales del synth
        else:
            freq= freqsMidi[midiNote]
            self.channels[midiNote]= WaveformSynthFM(
                    fc=freq,
                    amp=self.ampS.get(), ratio=self.ratioS.get(), beta=self.betaS.get(),
                    attack = self.attackS.get(), decay= self.decayS.get(),
                    sustain=self.sustainS.get(), release=self.sustainS.get(),carrier=self.carrierCB.get(), mod=self.modCB.get())

Writing waveformInstrument.py


In [3]:
# Ejercicio 3

import tkinter as tk
import numpy as np
import sounddevice as sd

CHUNK = 512
SRATE = 48000

class OscWaveTable:
    def __init__(self, size, frec, vol):
        self.frec = frec
        self.vol = vol
        self.size = size
        t = np.linspace(0, 1, num=size)
        self.waveTable = np.sin(2 * np.pi * t)
        self.fase = 0
        self.step = self.size/(SRATE/self.frec)
        
    def setFrec(self,frec):
        self.frec = frec
        self.step = self.size/(SRATE/self.frec)
      
    def getFrec(self):
         return self.frec 
    
    def setVol(self, vol):
        self.vol = vol

    def getVol(self):
        return self.vol
    
    def next(self):
        samples = np.zeros(CHUNK, dtype=np.float32)
        cont = 0
        while cont < CHUNK:
            self.fase = (self.fase + self.step) % self.size
            x0 = int(self.fase) % self.size
            x1 = (x0 + 1) % self.size
            y0, y1 = self.waveTable[x0], self.waveTable[x1]
            samples[cont] = y0 + (self.fase - x0) * (y1 - y0) / (x1 - x0)
            cont += 1
        return self.vol * samples

def update(event):
    global osc
    x, y = event.x, event.y
     # Frecuencia de 100 a 1500
    frec = 100 + (x / root.winfo_width()) * 1400 
    # Volumen de 0 a 1
    vol = 1 - (y / root.winfo_height())  
    osc.setVol(vol)
    osc.setFrec(frec)
    
def callback(outdata, frames, time, status):    
        if status: print(status)    
        s = np.sum([i.next() for i in inputs],axis=0)
        s = np.float32(s)
        outdata[:] = s.reshape(-1, 1)

osc = OscWaveTable(110,1,1024)
inputs = [osc]
stream = sd.OutputStream(samplerate=SRATE,blocksize=CHUNK,channels=1, callback=callback)
stream.start()

# Ventana Tkinter
root = tk.Tk()

root.bind('<Motion>', update)

root.mainloop()

stream.stop()
stream.close()